###### Name: Aritra Mondal 
###### Cohort Code: GN22DT005 
###### EMP Id: 2143323

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count,udf, max,round,avg, countDistinct, row_number, desc
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("flight").getOrCreate()

In [0]:
# df = spark.read.options(inferSchema='True', header='True', delimiter=',').csv('/FileStore/tables/2007_csv.bz2')
df = spark.read.csv("/FileStore/tables/2007_csv.bz2", inferSchema=True, header=True )

In [0]:
df.show(truncate=False)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2007|1    |1         |1        |1232   |1225      |1341   |1340      |WN           |2891     

In [0]:
df.columns

Out[87]: ['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'TailNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Origin',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay']

In [0]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- C

In [0]:
df.describe().show()

+-------+--------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------+------------------+-------+------------------+-----------------+------------------+------------------+------------------+-------+-------+-----------------+-----------------+------------------+--------------------+----------------+--------------------+------------------+------------------+------------------+-------------------+-----------------+
|summary|                Year|             Month|        DayofMonth|         DayOfWeek|          DepTime|        CRSDepTime|           ArrTime|        CRSArrTime|UniqueCarrier|         FlightNum|TailNum| ActualElapsedTime|   CRSElapsedTime|           AirTime|          ArrDelay|          DepDelay| Origin|   Dest|         Distance|           TaxiIn|           TaxiOut|           Cancelled|CancellationCode|            Diverted|      CarrierDelay|      WeatherDelay|          NA

In [0]:
df.count()

Out[89]: 7453215

# 1. Find the most frequent tail number which is getting in destination by maximum

In [0]:
df1 = df.select("Dest", "TailNum")
df1 = df1.filter(df1.Dest.isNotNull() & df1.TailNum.isNotNull())
df1.show()

+----+-------+
|Dest|TailNum|
+----+-------+
| ONT|   N351|
| PDX|   N370|
| PDX|   N685|
| PDX|   N364|
| PDX|   N480|
| PDX| N611SW|
| PHX|   N482|
| PHX| N749SW|
| PHX|   N451|
| PHX| N630WN|
| PHX|   N493|
| PHX| N733SW|
| SAN|   N348|
| SAN|   N358|
| SAN|   N413|
| SAN|   N416|
| SAN| N383SW|
| SAN| N316SW|
| SAN| N691WN|
| SAN| N308SW|
+----+-------+
only showing top 20 rows



In [0]:
result = df1.groupBy("Dest", "TailNum").count().orderBy("count", ascending=False)
result.show()

+----+-------+-----+
|Dest|TailNum|count|
+----+-------+-----+
| ORD|      0|11709|
| DFW|      0| 9361|
| EWR|      0| 5194|
| LGA|      0| 3935|
| ORD| 000000| 3201|
| JFK|      0| 3123|
| BOS|      0| 2736|
| LAX|      0| 2274|
| HNL| N655BR| 2241|
| HNL| N651BR| 2173|
| HNL| N654BR| 2138|
| DTW|      0| 2108|
| HNL| N693BR| 2067|
| DCA|      0| 2062|
| HNL| N479HA| 2038|
| HNL| N478HA| 2024|
| HNL| N485HA| 1984|
| HNL| N480HA| 1976|
| IAH|      0| 1967|
| HNL| N484HA| 1944|
+----+-------+-----+
only showing top 20 rows



##### Filtering values where TailNum = 0

In [0]:
result.filter(~ result["TailNum"].startswith("0")).show()

+----+-------+-----+
|Dest|TailNum|count|
+----+-------+-----+
| HNL| N655BR| 2241|
| HNL| N651BR| 2173|
| HNL| N654BR| 2138|
| HNL| N693BR| 2067|
| HNL| N479HA| 2038|
| HNL| N478HA| 2024|
| HNL| N485HA| 1984|
| HNL| N480HA| 1976|
| HNL| N484HA| 1944|
| HNL| N487HA| 1909|
| HNL| N481HA| 1868|
| HNL| N810AL| 1843|
| HNL| N477HA| 1837|
| HNL| N837AL| 1836|
| HNL| N475HA| 1816|
| HNL| N486HA| 1787|
| HNL| N646BR| 1768|
| HNL| N836AL| 1761|
| HNL| N824AL| 1754|
| HNL| N808AL| 1740|
+----+-------+-----+
only showing top 20 rows



# 2. Find out the cancelled flight  details for the last quarter of the year 2007

In [0]:
df2 = df.select("FlightNum","Month", 'Cancelled')
df2.show()

+---------+-----+---------+
|FlightNum|Month|Cancelled|
+---------+-----+---------+
|     2891|    1|        0|
|      462|    1|        0|
|     1229|    1|        0|
|     1355|    1|        0|
|     2278|    1|        0|
|     2386|    1|        0|
|      409|    1|        0|
|     1131|    1|        0|
|     1212|    1|        0|
|     2456|    1|        0|
|     2575|    1|        0|
|     2608|    1|        0|
|      139|    1|        0|
|      747|    1|        0|
|      933|    1|        0|
|     1054|    1|        0|
|     1106|    1|        0|
|     1554|    1|        0|
|     1604|    1|        0|
|     1975|    1|        0|
+---------+-----+---------+
only showing top 20 rows



In [0]:
result = df2.filter((df2["Month"].isin([10,11,12])) & (df2["Cancelled"] == 1))

In [0]:
print(result.count())
result.show()

35099
+---------+-----+---------+
|FlightNum|Month|Cancelled|
+---------+-----+---------+
|      195|   10|        1|
|      710|   10|        1|
|     1191|   10|        1|
|      514|   10|        1|
|     1799|   10|        1|
|       30|   10|        1|
|      507|   10|        1|
|      449|   10|        1|
|      820|   10|        1|
|     1036|   10|        1|
|     1243|   10|        1|
|     1804|   10|        1|
|     2010|   10|        1|
|     1235|   10|        1|
|     1970|   10|        1|
|      336|   10|        1|
|      507|   10|        1|
|     2843|   10|        1|
|      162|   10|        1|
|     2980|   10|        1|
+---------+-----+---------+
only showing top 20 rows



# 3. Find out the average weather delays for a particular flight per month

In [0]:
df3 = df.select('FlightNum', 'Month', 'WeatherDelay')

In [0]:
result = df3.groupBy('FlightNum', 'Month').avg('WeatherDelay').orderBy('FlightNum', 'Month')

In [0]:
result = result.withColumnRenamed("avg(WeatherDelay)", "Average_Weather_Delay")

In [0]:
result.show()

+---------+-----+---------------------+
|FlightNum|Month|Average_Weather_Delay|
+---------+-----+---------------------+
|        1|    1|  0.40476190476190477|
|        1|    2|   3.0197368421052633|
|        1|    3|                 0.89|
|        1|    4|   0.5185185185185185|
|        1|    5|  0.17829457364341086|
|        1|    6|   0.1589958158995816|
|        1|    7|   0.8492871690427699|
|        1|    8|   0.7018255578093306|
|        1|    9|   0.2597938144329897|
|        1|   10|   0.9749518304431599|
|        1|   11|   0.3134715025906736|
|        1|   12|   0.8363636363636363|
|        2|    1|  0.05555555555555555|
|        2|    2|  0.21923076923076923|
|        2|    3|  0.18055555555555555|
|        2|    4|  0.07547169811320754|
|        2|    5| 0.050397877984084884|
|        2|    6| 0.027548209366391185|
|        2|    7|   0.0427807486631016|
|        2|    8|   0.4037940379403794|
+---------+-----+---------------------+
only showing top 20 rows



# 4. Inspite of NASDelay, SecurityDelay, LateAircraftDelay,Weatherdealy which flight reached on time

In [0]:
df4 = df.select('FlightNum', 'CarrierDelay', 'WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay', 'ArrDelay')

In [0]:
df4 = df4.dropna()

In [0]:
df4.show()

+---------+------------+------------+--------+-------------+-----------------+--------+
|FlightNum|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|ArrDelay|
+---------+------------+------------+--------+-------------+-----------------+--------+
|     2891|           0|           0|       0|            0|                0|       1|
|      462|           0|           0|       0|            0|                0|       8|
|     1229|           3|           0|       0|            0|               31|      34|
|     1355|          23|           0|       0|            0|                3|      26|
|     2278|           0|           0|       0|            0|                0|      -3|
|     2386|           0|           0|       0|            0|                0|       3|
|      409|          46|           0|       0|            0|                1|      47|
|     1131|           0|           0|       0|            0|                0|      -2|
|     1212|          20|        

In [0]:
df4.count()

Out[119]: 7453215

In [0]:
df4.filter((df['CarrierDelay'] > 0) &
           (df['WeatherDelay'] > 0) &
           (df['NASDelay'] > 0) &
           (df['SecurityDelay'] > 0) &
           (df['LateAircraftDelay'] > 0) &
           (df['ArrDelay'] <= 0)).show()

+---------+------------+------------+--------+-------------+-----------------+--------+
|FlightNum|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|ArrDelay|
+---------+------------+------------+--------+-------------+-----------------+--------+
+---------+------------+------------+--------+-------------+-----------------+--------+



In [0]:
df4.filter(((df['CarrierDelay'] > 0) |
           (df['WeatherDelay'] > 0) |
           (df['NASDelay'] > 0) |
           (df['SecurityDelay'] > 0) |
           (df['LateAircraftDelay'] > 0)) & 
           (df['ArrDelay'] <= 0)).show()

+---------+------------+------------+--------+-------------+-----------------+--------+
|FlightNum|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|ArrDelay|
+---------+------------+------------+--------+-------------+-----------------+--------+
+---------+------------+------------+--------+-------------+-----------------+--------+



# 5. Month wise total distance travelled by each flight number in every month

In [0]:
df5 = df.select('FlightNum', 'Month', 'Distance')
df5.show()

+---------+-----+--------+
|FlightNum|Month|Distance|
+---------+-----+--------+
|     2891|    1|     389|
|      462|    1|     479|
|     1229|    1|     479|
|     1355|    1|     479|
|     2278|    1|     479|
|     2386|    1|     479|
|      409|    1|     647|
|     1131|    1|     647|
|     1212|    1|     647|
|     2456|    1|     647|
|     2575|    1|     647|
|     2608|    1|     647|
|      139|    1|     480|
|      747|    1|     480|
|      933|    1|     480|
|     1054|    1|     480|
|     1106|    1|     480|
|     1554|    1|     480|
|     1604|    1|     480|
|     1975|    1|     480|
+---------+-----+--------+
only showing top 20 rows



In [0]:
result = df5.groupBy('FlightNum', 'Month').sum('Distance').withColumnRenamed("sum(Distance)", "Total Distance Travelled").orderBy('FlightNum', 'Month').show()

+---------+-----+------------------------+
|FlightNum|Month|Total Distance Travelled|
+---------+-----+------------------------+
|        1|    1|                  627051|
|        1|    2|                  566492|
|        1|    3|                  661394|
|        1|    4|                  705351|
|        1|    5|                  749570|
|        1|    6|                  721426|
|        1|    7|                  744727|
|        1|    8|                  745205|
|        1|    9|                  716980|
|        1|   10|                  748800|
|        1|   11|                  677085|
|        1|   12|                  739739|
|        2|    1|                  590173|
|        2|    2|                  533012|
|        2|    3|                  590518|
|        2|    4|                  617762|
|        2|    5|                  661826|
|        2|    6|                  637585|
|        2|    7|                  659483|
|        2|    8|                  646500|
+---------+

# 6. Month wise how many flights get diverted(origin to destination)

In [0]:
df6 = df.select('FlightNum', 'Month', 'Diverted')
df6.show()

+---------+-----+--------+
|FlightNum|Month|Diverted|
+---------+-----+--------+
|     2891|    1|       0|
|      462|    1|       0|
|     1229|    1|       0|
|     1355|    1|       0|
|     2278|    1|       0|
|     2386|    1|       0|
|      409|    1|       0|
|     1131|    1|       0|
|     1212|    1|       0|
|     2456|    1|       0|
|     2575|    1|       0|
|     2608|    1|       0|
|      139|    1|       0|
|      747|    1|       0|
|      933|    1|       0|
|     1054|    1|       0|
|     1106|    1|       0|
|     1554|    1|       0|
|     1604|    1|       0|
|     1975|    1|       0|
+---------+-----+--------+
only showing top 20 rows



In [0]:
df6.filter(df['Diverted'] == 1).groupBy('Month').agg(count('FlightNum').alias('Total no. of diverted flights')).show()

+-----+-----------------------------+
|Month|Total no. of diverted flights|
+-----+-----------------------------+
|    1|                         1200|
|    2|                         1261|
|    3|                         1275|
|    4|                         1193|
|    5|                         1442|
|    6|                         2199|
|    7|                         2150|
|    8|                         2101|
|    9|                          962|
|   10|                         1000|
|   11|                          881|
|   12|                         1515|
+-----+-----------------------------+



# 7. Week and month wise number of trips in all the flights

In [0]:
df7 = df.select('DayofMonth', 'Month','FlightNum').where(df.Cancelled == 0)
df7.show()

+----------+-----+---------+
|DayofMonth|Month|FlightNum|
+----------+-----+---------+
|         1|    1|     2891|
|         1|    1|      462|
|         1|    1|     1229|
|         1|    1|     1355|
|         1|    1|     2278|
|         1|    1|     2386|
|         1|    1|      409|
|         1|    1|     1131|
|         1|    1|     1212|
|         1|    1|     2456|
|         1|    1|     2575|
|         1|    1|     2608|
|         1|    1|      139|
|         1|    1|      747|
|         1|    1|      933|
|         1|    1|     1054|
|         1|    1|     1106|
|         1|    1|     1554|
|         1|    1|     1604|
|         1|    1|     1975|
+----------+-----+---------+
only showing top 20 rows



In [0]:
def week(x):
    if(x >=1 and x <=7):
        return 1
    elif(x >= 8 and x <= 14):
        return 2
    elif(x >= 15 and x <= 21):
        return 3
    elif(x >= 22 and x <= 28):
        return 4
    else:
        return 5

In [0]:
day_to_week = udf(lambda x : week(x))

In [0]:
df7 = df7.withColumn("Week", day_to_week(df7["DayofMonth"]))
df7.show()

+----------+-----+---------+----+
|DayofMonth|Month|FlightNum|Week|
+----------+-----+---------+----+
|         1|    1|     2891|   1|
|         1|    1|      462|   1|
|         1|    1|     1229|   1|
|         1|    1|     1355|   1|
|         1|    1|     2278|   1|
|         1|    1|     2386|   1|
|         1|    1|      409|   1|
|         1|    1|     1131|   1|
|         1|    1|     1212|   1|
|         1|    1|     2456|   1|
|         1|    1|     2575|   1|
|         1|    1|     2608|   1|
|         1|    1|      139|   1|
|         1|    1|      747|   1|
|         1|    1|      933|   1|
|         1|    1|     1054|   1|
|         1|    1|     1106|   1|
|         1|    1|     1554|   1|
|         1|    1|     1604|   1|
|         1|    1|     1975|   1|
+----------+-----+---------+----+
only showing top 20 rows



In [0]:
df7.groupBy('Month', 'Week').agg(count('FlightNum').alias("No. of flights")).orderBy('Month', 'Week').show()

+-----+----+--------------+
|Month|Week|No. of flights|
+-----+----+--------------+
|    1|   1|        139114|
|    1|   2|        134874|
|    1|   3|        133690|
|    1|   4|        137875|
|    1|   5|         60229|
|    2|   1|        134924|
|    2|   2|        129914|
|    2|   3|        138929|
|    2|   4|        136372|
|    3|   1|        139401|
|    3|   2|        142861|
|    3|   3|        138140|
|    3|   4|        142412|
|    3|   5|         59518|
|    4|   1|        142217|
|    4|   2|        140864|
|    4|   3|        140149|
|    4|   4|        139503|
|    4|   5|         40777|
|    5|   1|        141346|
+-----+----+--------------+
only showing top 20 rows



# 8. Which flights covered maximum origin and destination by month wise

In [0]:
df8 = df.select('FlightNum', 'Month','Origin','Dest','Cancelled')
df8.show()

+---------+-----+------+----+---------+
|FlightNum|Month|Origin|Dest|Cancelled|
+---------+-----+------+----+---------+
|     2891|    1|   SMF| ONT|        0|
|      462|    1|   SMF| PDX|        0|
|     1229|    1|   SMF| PDX|        0|
|     1355|    1|   SMF| PDX|        0|
|     2278|    1|   SMF| PDX|        0|
|     2386|    1|   SMF| PDX|        0|
|      409|    1|   SMF| PHX|        0|
|     1131|    1|   SMF| PHX|        0|
|     1212|    1|   SMF| PHX|        0|
|     2456|    1|   SMF| PHX|        0|
|     2575|    1|   SMF| PHX|        0|
|     2608|    1|   SMF| PHX|        0|
|      139|    1|   SMF| SAN|        0|
|      747|    1|   SMF| SAN|        0|
|      933|    1|   SMF| SAN|        0|
|     1054|    1|   SMF| SAN|        0|
|     1106|    1|   SMF| SAN|        0|
|     1554|    1|   SMF| SAN|        0|
|     1604|    1|   SMF| SAN|        0|
|     1975|    1|   SMF| SAN|        0|
+---------+-----+------+----+---------+
only showing top 20 rows



In [0]:
df8 = df8.dropna()

In [0]:
df8 = df8.filter(df.Cancelled == 0)

In [0]:
df_max_origin = df8.groupBy('Month', 'FlightNum').agg(countDistinct('Origin').alias("Max Visited Origins")).sort('Month', desc('Max Visited Origins'))
windowDept = Window.partitionBy("Month").orderBy(col('Max Visited Origins').desc())
df_max_origin.withColumn("row",row_number().over(windowDept)) \
  .filter(col("row") == 1).drop("row") \
  .show()

+-----+---------+-------------------+
|Month|FlightNum|Max Visited Origins|
+-----+---------+-------------------+
|    1|      433|                 18|
|    2|      500|                 18|
|    3|      644|                 18|
|    4|      644|                 17|
|    5|      644|                 17|
|    6|      226|                 18|
|    7|      425|                 17|
|    8|       67|                 18|
|    9|       62|                 20|
|   10|       66|                 20|
|   11|      303|                 21|
|   12|      151|                 22|
+-----+---------+-------------------+



In [0]:
df_max_dest = df8.groupBy('Month', 'FlightNum').agg(countDistinct('Dest').alias("Max Visited Destinations")).sort('Month', desc('Max Visited Destinations'))
windowDept = Window.partitionBy("Month").orderBy(col('Max Visited Destinations').desc())
df_max_dest.withColumn("row",row_number().over(windowDept)) \
  .filter(col("row") == 1).drop("row") \
  .show()

+-----+---------+------------------------+
|Month|FlightNum|Max Visited Destinations|
+-----+---------+------------------------+
|    1|      372|                      18|
|    2|      432|                      17|
|    3|      432|                      17|
|    4|      473|                      17|
|    5|      644|                      19|
|    6|      308|                      18|
|    7|      425|                      17|
|    8|       67|                      18|
|    9|      385|                      18|
|   10|       66|                      20|
|   11|      303|                      21|
|   12|      151|                      20|
+-----+---------+------------------------+



In [0]:
df8.groupBy('Month','Origin','Dest','FlightNum').count().sort("count", ascending=False).withColumnRenamed("count", "distCount").dropDuplicates(['Month']).sort("Month").show()

+-----+------+----+---------+---------+
|Month|Origin|Dest|FlightNum|distCount|
+-----+------+----+---------+---------+
|    1|   SFO| JFK|       16|       62|
|    2|   LIH| HNL|      110|       56|
|    3|   ORD| SAN|      309|       62|
|    4|   LAX| JFK|       30|       60|
|    5|   SFO| SEA|      295|       62|
|    6|   LAX| JFK|       30|       59|
|    7|   SAN| PHX|      614|       62|
|    8|   JFK| SFO|       15|       62|
|    9|   JFK| LAX|      133|       60|
|   10|   JFK| LAX|      133|       62|
|   11|   SFO| JFK|       16|       60|
|   12|   ATL| DFW|     1209|       62|
+-----+------+----+---------+---------+



# 9.Average month wise arrival delay (flightnum wise)

In [0]:
df9 = df.select('FlightNum', 'Month', 'ArrDelay')
df9.show()

+---------+-----+--------+
|FlightNum|Month|ArrDelay|
+---------+-----+--------+
|     2891|    1|       1|
|      462|    1|       8|
|     1229|    1|      34|
|     1355|    1|      26|
|     2278|    1|      -3|
|     2386|    1|       3|
|      409|    1|      47|
|     1131|    1|      -2|
|     1212|    1|      44|
|     2456|    1|      -7|
|     2575|    1|     -11|
|     2608|    1|      52|
|      139|    1|      45|
|      747|    1|     -17|
|      933|    1|      -5|
|     1054|    1|      33|
|     1106|    1|      -9|
|     1554|    1|      -7|
|     1604|    1|     -11|
|     1975|    1|      36|
+---------+-----+--------+
only showing top 20 rows



In [0]:
df9.groupBy('FlightNum', 'Month').agg(round(avg('ArrDelay'),3).alias("Average arrival delay")).orderBy('FlightNum', 'Month').show()

+---------+-----+---------------------+
|FlightNum|Month|Average arrival delay|
+---------+-----+---------------------+
|        1|    1|               10.716|
|        1|    2|               18.306|
|        1|    3|                8.578|
|        1|    4|                1.684|
|        1|    5|                3.216|
|        1|    6|                9.654|
|        1|    7|                6.444|
|        1|    8|                8.574|
|        1|    9|                1.266|
|        1|   10|                7.056|
|        1|   11|               -1.466|
|        1|   12|                8.183|
|        2|    1|               10.943|
|        2|    2|                 6.04|
|        2|    3|                9.852|
|        2|    4|                2.574|
|        2|    5|                5.276|
|        2|    6|               15.546|
|        2|    7|               10.869|
|        2|    8|                9.385|
+---------+-----+---------------------+
only showing top 20 rows



# 10.Average month wise departure delay (flightnum wise)

In [0]:
df10 = df.select('FlightNum', 'Month', 'DepDelay')
df10.show()

+---------+-----+--------+
|FlightNum|Month|DepDelay|
+---------+-----+--------+
|     2891|    1|       7|
|      462|    1|      13|
|     1229|    1|      36|
|     1355|    1|      30|
|     2278|    1|       1|
|     2386|    1|      10|
|      409|    1|      56|
|     1131|    1|       9|
|     1212|    1|      47|
|     2456|    1|       3|
|     2575|    1|       1|
|     2608|    1|      52|
|      139|    1|      53|
|      747|    1|      -5|
|      933|    1|       6|
|     1054|    1|      44|
|     1106|    1|       0|
|     1554|    1|       2|
|     1604|    1|       1|
|     1975|    1|      29|
+---------+-----+--------+
only showing top 20 rows



In [0]:
df10.groupBy('FlightNum', 'Month').agg(round(avg('DepDelay'),3).alias("Average departure delay")).orderBy('FlightNum', 'Month').show()

+---------+-----+-----------------------+
|FlightNum|Month|Average departure delay|
+---------+-----+-----------------------+
|        1|    1|                   9.85|
|        1|    2|                  16.93|
|        1|    3|                 11.173|
|        1|    4|                  5.615|
|        1|    5|                  3.711|
|        1|    6|                 12.008|
|        1|    7|                  9.791|
|        1|    8|                 10.459|
|        1|    9|                  4.576|
|        1|   10|                  9.892|
|        1|   11|                  3.935|
|        1|   12|                 10.609|
|        2|    1|                 12.219|
|        2|    2|                  8.558|
|        2|    3|                  7.493|
|        2|    4|                  3.601|
|        2|    5|                   3.96|
|        2|    6|                  11.51|
|        2|    7|                  8.065|
|        2|    8|                  9.328|
+---------+-----+-----------------